In [ ]:
!pip install simpletransformers
!mkdir -p 'drive/MyDrive/emotion_classifier'

In [1]:
import pandas as pd
import torch
import pickle
from torch.utils.data import Dataset, DataLoader # for the dataloader
from simpletransformers.classification import ClassificationModel, ClassificationArgs
from transformers import Trainer, TrainingArguments
from sklearn.metrics import classification_report, confusion_matrix, f1_score


# DataLoader

In [2]:
# Convert dataframe into dictionary of text and labels
def reader(df):
    texts = df['text'].values.tolist()
    labels = df['labels'].values.tolist()

    return {'texts':texts, 'labels':labels}

In [3]:
# DataLoader
class OlidDataset(Dataset):
  def __init__(self, tokenizer, input_set):
    # input_set: dictionary version of the df
    self.texts = input_set['texts']
    self.labels = input_set['labels']
    self.tokenizer = tokenizer

  def collate_fn(self, batch):
    texts = []
    labels = []

    for b in batch:
      texts.append(str(b['text']))
      labels.append(b['label'])

    print(texts)
    print(labels)
    encodings = self.tokenizer(
      texts,                        # what to encode
      return_tensors = 'pt',        # return pytorch tensors
      add_special_tokens = True,    # incld tokens like [SEP], [CLS]
      padding = "max_length",       # pad to max sentence length
      truncation = True,            # truncate if too long
      max_length= 128)              

    encodings['labels'] = torch.tensor(labels)
    return encodings

  def __len__(self):
    return len(self.texts)

  def __getitem__(self, idx):
    item = {'text': self.texts[idx], 'label': self.labels[idx]}

    return item


# Train Model

In [7]:
def train_model(model_name, best_model_dir, train_df, eval_df):
  optimizer = 'AdamW' 
  learning_rate = 4e-05
  epochs = 1
  
  model_args = ClassificationArgs(num_train_epochs=epochs,        # number of epochs
                                  best_model_dir=best_model_dir,  # directory to save best model
                                  evaluate_during_training=True,  # best model determined by validation set performance
                                  no_cache=True,                  
                                  save_steps=-1,                  
                                  save_model_every_epoch=False,
                                  overwrite_output_dir=True,
                                  learning_rate=learning_rate,    # learning rate
                                  optimizer=optimizer)            # optimizer

  model = ClassificationModel(model_type="xlmroberta",  # tried xlmroberta, bert
                            model_name=model_name,      # tried bert-base-chinese, xlm-roberta-base, bert-base-multilingual-cased (mBert), microsoft/infoxlm-base
                            args = model_args,          # see above
                            num_labels=4,               # 4 labels - sad, happy, fear, anger
                            use_cuda=cuda_available)    # use GPU
  
  if model_name != 'xlm-roberta-base':
    print('Sanity Check, make sure training correct model.')
    evaluate(model, df_val_EP)

  model.train_model(train_df = train_df,                # training dataset
                    eval_df = eval_df)                  # evaluation dataset
  
  return model


# Evaluate

In [5]:
def evaluate(model, df_dataset):
  y_pred, _ = model.predict(df_dataset.text.tolist())
  y_true = df_dataset['labels']

  print("Classification Report", classification_report(y_true, y_pred))
  print("Confusion Matrix", confusion_matrix(y_true, y_pred))
  print("F1-Score", f1_score(y_true, y_pred,average='weighted'))

# Run Code

In [ ]:
# Run
GPU = True
if GPU:
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
else:
    device = torch.device("cpu")
print(f"Using {device}")

cuda_available = torch.cuda.is_available()

## Datasets
# Emotion (Twitter) Dataset (First Tune)
df_train_twitter = pd.read_csv('/content/drive/MyDrive/emotion_classifier/twitter_full.csv')

# EmpatheticPersonas (EP) Dataset (Second Tune)
df_train_EP = pd.read_csv('/content/drive/MyDrive/emotion_classifier/emotionlabeled_train.csv')
df_val_EP = pd.read_csv('/content/drive/MyDrive/emotion_classifier/emotionlabeled_val.csv')
df_test_EP = pd.read_csv('/content/drive/MyDrive/emotion_classifier/emotionlabeled_val.csv')

## Begin Finetune
# First Finetune 
model = train_model(model_name = "xlm-roberta-base",
                    best_model_dir = "/content/drive/MyDrive/emotion_classifier/best_finetuned_1/",
                    train_df = df_train_twitter[['text','labels']],
                    eval_df = df_val_EP[['text','labels']])

# Evaluate first finetune
print('Performance after First Finetune (Twitter) on Validation Set')
evaluate(model, df_val_EP)

# Second Finetune
model = train_model(model_name = "/content/drive/MyDrive/emotion_classifier/best_finetuned_1",
                    best_model_dir = "/content/drive/MyDrive/emotion_classifier/best_finetuned_2/",
                    train_df = df_train_EP[['text','labels']],
                    eval_df = df_val_EP[['text','labels']])

# Evaluate second finetune
print('Performance after Second Finetune (EP) on Validation Set')
evaluate(model, df_val_EP)

# Final Test results
print('Performance on Held-Out Test Set')
evaluate(model, df_test_EP)
